# CX2Network Tutorial

In this tutorial you will learn to use `CX2Network`, a simple data model that is part of the `ndex2 NDEx Client` module.
`CX2Network` facilitates creating and working with networks, including interfaces to NetworkX and Pandas.
This tutorial requires Python 3.8+ and the ndex2 module, see the NDEx Client Tutorial for installation instructions.

### Import Packages Required for this Tutorial

In [ ]:
from ndex2.cx2 import CX2Network, RawCX2NetworkFactory, NetworkXToCX2NetworkFactory, PandasDataFrameToCX2NetworkFactory, CX2NetworkPandasDataFrameFactory, CX2NetworkXFactory 
import ndex2.client as nc
import ndex2
import networkx as nx
import pandas as pd
import os
import json
import ndex2.constants as constants

# Five ways to create CX2 networks:

- Starting with an Empty Network
- CX2 Files
- NDEx Networks
- NetworkX Networks
- Pandas DataFrames

For information on **how to access nodes, edges, attributes, etc...** see the end of this notebook ([click here](#first-bullet))

## Starting with an Empty Network

**Create an empty CX2 network**

In [ ]:
cx2_creatures = CX2Network()

**Name the Network**

The _cx2_creatures_ will now be populated with data in which each node represents a species and has a color attribute. Each edge will specify a relationship between the two species. First, we will set the name of the network:  

In [ ]:
cx2_creatures.add_network_attribute("name", "Food Web")

**Add Nodes and Edges**

In [ ]:
fox_node = cx2_creatures.add_node(attributes={'name': 'Fox'})
mouse_node = cx2_creatures.add_node(attributes={'name': 'Mouse'})
bird_node = cx2_creatures.add_node(attributes={'name': 'Bird'})
fox_bird_edge = cx2_creatures.add_edge(source=fox_node, target=bird_node, attributes={'interaction': 'interacts-with'})

fox_mouse_edge = cx2_creatures.add_edge(source=fox_node, target=mouse_node, attributes={'interaction': 'interacts-with'})

print('The fox node has id: %d' % fox_node)

print('The mouse node has id: %d' % mouse_node)


The add_node() and add_edge() methods return the unique ID assigned to the new node or edge. In CX2, IDs are assigned in an ascending order unless specified otherwise by user. In this case, the node with the name "Fox" will have an ID of 0, "Mouse" will have 1, and so on. The edge between "Fox" and "Bird" will have the ID of 0, the next will be 1, and so on. 

**Add Attributes**

The add_node_attribute() and add_edge_attribute() methods require the ID in the "property_of" field and the property "name" and "values" in their respective fields.

In [ ]:
print(cx2_creatures)
cx2_creatures.add_node_attribute(fox_node, key='Color', value='Red')

cx2_creatures.add_node_attribute(mouse_node, key='Color', value='Gray')

cx2_creatures.add_node_attribute(bird_node, key='Color', value='Blue')

In [ ]:
cx2_creatures.add_edge_attribute(fox_mouse_edge, key='Hunted', value='On the ground')

We can now print network attributes, nodes and edges of cx2_creatures.

In [ ]:
print('network_attributes:', cx2_creatures.get_network_attributes())
print('nodes:', cx2_creatures.get_nodes())
print('edges:', cx2_creatures.get_edges())

## CX2 Files

In [ ]:
# Create CX2Network factory
factory = RawCX2NetworkFactory()

cx2_from_cx_file = factory.get_cx2network('SimpleNetwork.cx2')

print('network_attributes:', cx2_from_cx_file.get_network_attributes())
print('nodes:', cx2_from_cx_file.get_nodes())
print('edges:', cx2_from_cx_file.get_edges())

## NDEx Networks

In [ ]:
network_uuid='f1dd6cc3-0007-11e6-b550-06603eb7f303'

# Create NDEx2 python client
client = ndex2.client.Ndex2(host='ndexbio.org')

# Create CX2Network factory
factory = RawCX2NetworkFactory()

# Get network as cx2 stream
client_resp = client.get_network_as_cx2_stream(network_uuid)

# Convert downloaded network to CX2Network object
cx2_from_server = factory.get_cx2network(json.loads(client_resp.content))

print('network_attributes:', cx2_from_server.get_network_attributes())
print('nodes:', len(cx2_from_server.get_nodes()))
print('edges:', len(cx2_from_server.get_edges()))

## NetworkX Networks

Networks can be manipulated using NetworkX facilities and then used to create a CX2 network. Here we create a NetworkX network from scratch:

In [ ]:
G = nx.Graph()
G.add_node(1, name='ABC')
G.add_node(2, name='DEF')
G.add_node(3, name='GHI')
G.add_node(4, name='JKL')
G.add_node(5, name='MNO')
G.add_node(6, name='PQR')
G.add_node(7, name='XYZ')
G.add_edges_from([(1,2), (2, 3),(3, 4), 
                  (2, 4), (4, 5), (2, 5),
                  (5, 7), (2, 6)])

We now use NetworkX to find the shortest path between nodes ABC and MNO and then create a subgraph from that path.

In [ ]:
short_path = nx.shortest_path(G,source=1,target=5)

path_subgraph = G.subgraph(short_path)

Finally, CX2 objects are created from both *G* and *path_subgraph*.

In [ ]:
networkx_factory = NetworkXToCX2NetworkFactory()

G.name = 'Created from NetworkX (full)'
cx2_full = networkx_factory.get_cx2network(G)

G.name = 'Created from NetworkX (shortest path)'
cx2_short = networkx_factory.get_cx2network(path_subgraph)
                       
print('network_attributes:', cx2_full.get_network_attributes())
print('nodes:', len(cx2_full.get_nodes()))
print('edges:', len(cx2_full.get_edges()))
print(G.edges())
print('')

print('network_attributes:', cx2_short.get_network_attributes())
print('nodes:', len(cx2_short.get_nodes()))
print('edges:', len(cx2_short.get_edges()))
print(path_subgraph.edges())

## Pandas DataFrames

### 2 Column DataFrame with No Headers:

In [ ]:
data = [('ABC', 'DEF'), ('DEF', 'XYZ')]

df = pd.DataFrame.from_records(data)

pd_factory = PandasDataFrameToCX2NetworkFactory()

cx2_df_2_column = pd_factory.get_cx2network(df, source_field=0, target_field=1)

print('network_attributes:', cx2_df_2_column.get_network_attributes())
print('nodes:', len(cx2_df_2_column.get_nodes()))
print('edges:', len(cx2_df_2_column.get_edges()))

#===========================
# CONVERT BACK TO DATAFRAME
#===========================

cx2_pd_factory = CX2NetworkPandasDataFrameFactory()
df_2_col_from_cx2 = cx2_pd_factory.get_dataframe(cx2_df_2_column)
df_2_col_from_cx2

### 3 Column DataFrame with No Headers:

In [ ]:
data = [('ABC', 'DEF', 'interacts-with'), ('DEF', 'XYZ', 'neighbor-of')]

df = pd.DataFrame.from_records(data)

# Rename 3rd column for it to be considered as edge interaction
df.rename(columns={2: 'interaction'}, inplace=True)

cx2_df_3_column = pd_factory.get_cx2network(df, source_field=0, target_field=1)

print('network_attributes:', cx2_df_3_column.get_network_attributes())
print('nodes:', len(cx2_df_3_column.get_nodes()))
print('edges:', len(cx2_df_3_column.get_edges()))

#===========================
# CONVERT BACK TO DATAFRAME
#===========================
df_3_col_from_cx2 = cx2_pd_factory.get_dataframe(cx2_df_3_column)
df_3_col_from_cx2

### 3+ Column DataFrame with Headers to Specify Attribute Columns

In [ ]:
df = pd.DataFrame.from_dict(dict([('source', ['ABC', 'DEF']),
                              ('target', ['DEF', 'XYZ']),
                              ('interaction', ['interacts-with', 'neighbor-of']),
                              ('edgeProp', ['Edge property 1', 'Edge property 2'])]))

cx2_df_with_headers = pd_factory.get_cx2network(df, source_field='source', target_field='target')

print('network_attributes:', cx2_df_with_headers.get_network_attributes())
print('nodes:', len(cx2_df_with_headers.get_nodes()))
print('edges:', len(cx2_df_with_headers.get_edges()))

#===========================
# CONVERT BACK TO DATAFRAME
#===========================
df_headers_from_cx2 = cx2_pd_factory.get_dataframe(cx2_df_with_headers)
df_headers_from_cx2

### Exporting a Larger and More Complex Network

More complex networks can be output to a Pandas DataFrame as well. In the following example we convert the example network we loaded from the server.

In [ ]:
#======================
# CONVERT TO DATAFRAME
#======================
cx2_from_server_df = cx2_pd_factory.get_dataframe(cx2_from_server)
print(cx2_from_server_df)

Likewise, networks can be output as a networkx graph.

In [ ]:
#=====================
# CONVERT TO NETWORKX
#=====================
networkx_factory = CX2NetworkXFactory()

cx2_from_server_to_netx = networkx_factory.get_graph(cx2_from_server)

print(cx2_from_server_to_netx)

**To continue with the tutorial, you must edit the following cell to replace the values of the ‘my_account’ and ‘my_password’ variables with a real NDEx account name and password.**

In [ ]:
my_account="enter your username here"
my_password="enter your password here"
my_server="http://ndexbio.org"
my_ndex = None
if my_account == 'enter your username here':
    print('*******WARNING!!!!*******')
    print('Please change the username and password before proceeding')
else:
    try:
        my_ndex=nc.Ndex2(my_server, my_account, my_password)
        my_ndex.update_status()
        print("Success.  Please continue.")
    except Exception as inst:
        print("Could not access account %s with password %s" % (my_account, my_password))
        print(inst.args)

CX2 networks can be saved to the NDEx server by calling **save_new_cx2_network()**

In [ ]:
if not my_ndex:
    print('*******WARNING!!!!*******')
    print('Please change the username and password before proceeding')
else:
    upload_message = my_ndex.save_new_cx2_network(cx2_df_with_headers.to_cx2(), visibility='PRIVATE')
    print(upload_message)

## Accessing nodes, edges, attributes, etc... <a class="anchor" id="first-bullet"></a>

### Nodes

In [ ]:
# EXAMPLE NETWORK FROM NDEx
uuid='f1dd6cc3-0007-11e6-b550-06603eb7f303'

# Create NDEx2 python client
client = ndex2.client.Ndex2(host='ndexbio.org')

# Create CX2Network factory
factory = RawCX2NetworkFactory()

# Get network as cx2 stream
client_resp = client.get_network_as_cx2_stream(uuid)

# Convert downloaded network to CX2Network object
cx2_network = factory.get_cx2network(json.loads(client_resp.content))

Loop through all **nodes** and print when we reach the RAD52 node

In [ ]:
for node_id, node in cx2_network.get_nodes().items():
    if node.get(constants.ASPECT_VALUES).get('name') == 'RAD52':
        print(node)
        break

Nodes and edges are indexed by id.  To look up a specific node by it's _name_ you use ``lookup_node_id_by_name`` method.

In [ ]:
#========================
# GET THE 'MAP2K1' NODE
#========================
map2k1_node_id = cx2_network.lookup_node_id_by_name( 'MAP2K1' )
map2k1_node = cx2_network.get_node(map2k1_node_id)
print(map2k1_node)

### Node attributes

Get the node attribute (Pathway) for MAP2K1

In [ ]:
map2k1_pathway_attribute = cx2_network.get_node(map2k1_node_id).get(constants.ASPECT_VALUES).get('Pathway')
print(map2k1_pathway_attribute)

### Edges

Print all the **edges** where MAP2K1 is either a source or target

In [ ]:
map2k1_edges = []

for edge_id, edge in cx2_network.get_edges().items():
    if edge.get('s') == map2k1_node_id or edge.get('t') == map2k1_node_id:
        map2k1_edges.append(edge.get('id'))

        print(edge)


Or, if you would like to see the node labels represented in these edges you can look up and print the node name

In [ ]:
for edge_id, edge in cx2_network.get_edges().items():
    if edge.get('s') == map2k1_node_id or edge.get('t') == map2k1_node_id:
        source_node = cx2_network.get_node( edge.get('s') )
        target_node = cx2_network.get_node( edge.get('t') )
        print('Source: %s Target: %s Interaction: %s' % (source_node.get(constants.ASPECT_VALUES).get('name'), target_node.get(constants.ASPECT_VALUES).get('name'), edge.get(constants.ASPECT_VALUES).get('interaction')))

### Edge attributes

Print the **edge attributes** (yeastSscore) for MAP2K1 edges

In [ ]:
for edge_id in map2k1_edges:
    print(cx2_network.get_edge(edge_id).get(constants.ASPECT_VALUES).get('yeastSscore'))


### Network attributes

Print the reference attribute for this network

In [ ]:
cx2_network.get_network_attributes().get('reference')

Or print the html content in the value field

In [ ]:
from IPython.core.display import display, HTML

display(HTML(cx2_network.get_network_attributes().get('reference')))